# Forest Cover Type – Training Notebook
Algorithms: Logistic Regression, SVM, MLP Neural Network.

## 1. Load Data

In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

df = pd.read_csv("covtype.csv")

X = df.drop(columns=["Cover_Type"])
y = df["Cover_Type"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## 2. Logistic Regression

In [2]:

lr = LogisticRegression(max_iter=2000, n_jobs=-1)
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_test)

print("Logistic Regression Accuracy:", accuracy_score(y_test, pred_lr))
print(classification_report(y_test, pred_lr))


Logistic Regression Accuracy: 0.7233117905733931
              precision    recall  f1-score   support

           1       0.71      0.70      0.70     42368
           2       0.75      0.80      0.77     56661
           3       0.68      0.80      0.73      7151
           4       0.61      0.43      0.51       549
           5       0.14      0.00      0.01      1899
           6       0.49      0.27      0.35      3473
           7       0.74      0.56      0.63      4102

    accuracy                           0.72    116203
   macro avg       0.59      0.51      0.53    116203
weighted avg       0.71      0.72      0.71    116203



## 3. Support Vector Machine (RBF)

In [3]:
from sklearn.svm import LinearSVC

svm = LinearSVC()
svm.fit(X_train, y_train)
pred_svm = svm.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, pred_svm))
print(classification_report(y_test, pred_svm))


SVM Accuracy: 0.7114188101856235
              precision    recall  f1-score   support

           1       0.71      0.68      0.69     42368
           2       0.74      0.80      0.76     56661
           3       0.61      0.87      0.72      7151
           4       0.62      0.20      0.30       549
           5       0.56      0.01      0.02      1899
           6       0.43      0.06      0.10      3473
           7       0.68      0.51      0.58      4102

    accuracy                           0.71    116203
   macro avg       0.62      0.45      0.46    116203
weighted avg       0.70      0.71      0.70    116203



## 4. Neural Network (MLPClassifier)

In [4]:

mlp = MLPClassifier(hidden_layer_sizes=(128,64), max_iter=500)
mlp.fit(X_train, y_train)
pred_mlp = mlp.predict(X_test)

print("MLP Accuracy:", accuracy_score(y_test, pred_mlp))
print(classification_report(y_test, pred_mlp))


c:\Users\Dev\.conda\envs\pytlearning\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLP Accuracy: 0.9064395927815977
              precision    recall  f1-score   support

           1       0.92      0.88      0.90     42368
           2       0.90      0.94      0.92     56661
           3       0.90      0.91      0.90      7151
           4       0.84      0.80      0.82       549
           5       0.82      0.67      0.74      1899
           6       0.82      0.84      0.83      3473
           7       0.95      0.89      0.92      4102

    accuracy                           0.91    116203
   macro avg       0.88      0.85      0.86    116203
weighted avg       0.91      0.91      0.91    116203

